In [24]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna
import os
import json 


conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
    SELECT DISTINCT
        f.match_date,
        f.team,
        f.is_home,
        f.gw,
        f.[is_promoted?],
        f.[is_early_season?],
        f.season,
        f.division,
        team_elo.elo as team_elo,
        f.team_rolling_summary_goals,
        f.team_rolling_summary_pens_made,
        f.team_rolling_summary_shots,
        f.team_rolling_summary_shots_on_target,
        f.team_rolling_summary_touches,
        f.team_rolling_summary_xg,
        f.team_rolling_summary_npxg,
        f.team_rolling_possession_touches_att_pen_area,
        f.team_rolling_keeper_psxg,
        f.team_rolling_conceded_summary_goals,
        f.team_rolling_conceded_summary_pens_made,
        f.team_rolling_conceded_summary_shots,
        f.team_rolling_conceded_summary_shots_on_target,
        f.team_rolling_conceded_summary_touches,
        f.team_rolling_conceded_summary_xg,
        f.team_rolling_conceded_summary_npxg,
        f.team_rolling_conceded_possession_touches_att_pen_area,
        f.team_rolling_conceded_keeper_psxg,
        f.opp_team,
        opp_elo.elo as opp_team_elo,
        f.opp_team_rolling_summary_goals,
        f.opp_team_rolling_summary_pens_made,
        f.opp_team_rolling_summary_shots,
        f.opp_team_rolling_summary_shots_on_target,
        f.opp_team_rolling_summary_touches,
        f.opp_team_rolling_summary_xg,
        f.opp_team_rolling_summary_npxg,
        f.opp_team_rolling_possession_touches_att_pen_area,
        f.opp_team_rolling_keeper_psxg,
        f.opp_team_rolling_conceded_summary_goals,
        f.opp_team_rolling_conceded_summary_pens_made,
        f.opp_team_rolling_conceded_summary_shots,
        f.opp_team_rolling_conceded_summary_shots_on_target,
        f.opp_team_rolling_conceded_summary_touches,
        f.opp_team_rolling_conceded_summary_xg,
        f.opp_team_rolling_conceded_summary_npxg,
        f.opp_team_rolling_conceded_possession_touches_att_pen_area,
        f.opp_team_rolling_conceded_keeper_psxg,
        ms.summary_goals as goals
    FROM 
        team_all_features_90_00200 f
    JOIN
        fbref_match_all_columns ms
            ON ms.match_url = f.match_url
            AND ms.team = f.team
    JOIN
        clubelo_features team_elo
            ON team_elo.fbref_team = f.team
                AND DATE(f.match_date) BETWEEN team_elo.start_date AND team_elo.end_date
    JOIN
        clubelo_features opp_elo
            ON opp_elo.fbref_team = f.opp_team
                AND DATE(f.match_date) BETWEEN opp_elo.start_date AND opp_elo.end_date
    WHERE 
        (f.team_rolling_summary_goals IS NOT NULL AND f.opp_team_rolling_summary_goals IS NOT NULL)
        AND f.division = 'Premier League'
                       """, conn)

df

,match_date,team,is_home,gw,is_promoted?,is_early_season?,season,division,team_elo,team_rolling_summary_goals,...,opp_team_rolling_conceded_summary_goals,opp_team_rolling_conceded_summary_pens_made,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,goals
0,2025-05-25 00:00:00,Tottenham,1,38,0,0,2024-2025,Premier League,1778.323242,0.811222,...,1.604323,0.189265,12.478522,4.128437,603.795044,1.482446,1.331035,26.528746,1.392191,1.0
1,2025-05-25 00:00:00,Brighton,0,38,0,0,2024-2025,Premier League,1809.790894,1.942928,...,2.199083,0.000000,15.406190,5.449610,593.018515,1.694755,1.694755,26.197995,2.080640,4.0
2,2025-05-25 00:00:00,Bournemouth,1,38,0,0,2024-2025,Premier League,1806.686401,1.221826,...,1.501753,0.165470,17.459047,4.194789,654.869872,1.860305,1.690205,28.673113,1.399942,2.0
3,2025-05-25 00:00:00,Leicester City,0,38,1,0,2024-2025,Premier League,1620.328247,1.052791,...,1.662430,0.099022,11.401362,4.082394,605.052197,1.304388,1.225170,23.430866,1.373387,0.0
4,2025-05-25 00:00:00,Newcastle Utd,1,38,0,0,2024-2025,Premier League,1878.094604,1.851561,...,0.968051,0.000000,11.809791,4.587958,650.468474,1.138399,1.138399,22.299200,1.289435,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,2020-09-12 00:00:00,Leeds United,0,1,1,1,2020-2021,Premier League,1624.624268,2.418047,...,1.436920,0.095730,7.266258,2.635221,496.180466,1.205448,1.128864,10.108849,1.160007,3.0
5316,2020-09-12 00:00:00,Crystal Palace,1,1,0,1,2020-2021,Premier League,1673.688843,0.570460,...,0.822600,0.000000,10.651285,3.048056,544.952012,1.216448,1.216448,20.837488,1.270361,1.0
5317,2020-09-12 00:00:00,Southampton,0,1,0,1,2020-2021,Premier League,1720.211182,1.923482,...,1.970332,0.000000,12.669481,4.397582,686.387950,1.478836,1.478836,20.791894,1.788755,0.0
5318,2020-09-12 00:00:00,Fulham,1,1,1,1,2020-2021,Premier League,1591.546265,1.364007,...,1.193816,0.199407,13.800604,4.952256,574.073749,1.695469,1.538354,24.615318,1.463729,0.0


In [26]:
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["team", "opp_team", "goals", "match_date"])
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)

y = df["goals"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'
df['match_date'] = pd.to_datetime(df['match_date'])

train_mask = df['match_date'] < cutoff_date
test_mask = df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [27]:
from sklearn.linear_model import PoissonRegressor
import numpy as np
from scipy.stats import poisson
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Fit Poisson regression
print("\nTraining Poisson Regression baseline...")
poisson_reg = PoissonRegressor(
    alpha=1.0,          # Regularization strength
    max_iter=1000,      # Increase if convergence issues
    fit_intercept=True
)

poisson_reg.fit(X_train_scaled, y_train)

# Make predictions
preds_poisson = poisson_reg.predict(X_test_scaled)

# Calculate traditional metrics
rmse_poisson = np.sqrt(mean_squared_error(y_test, preds_poisson))
mae_poisson = mean_absolute_error(y_test, preds_poisson)

# Calculate log-likelihood
def poisson_log_likelihood(y_true, y_pred):
    """
    Calculate log-likelihood for Poisson distribution
    y_true: actual goal counts
    y_pred: predicted lambda (rate parameter)
    """
    # Avoid log(0) by adding small epsilon
    y_pred = np.maximum(y_pred, 1e-10)
    
    # Poisson log-likelihood: y*log(λ) - λ - log(y!)
    log_likelihood = np.sum(
        y_true * np.log(y_pred) - y_pred - np.log(np.maximum(1, np.arange(1, len(y_true) + 1)))
    )
    return log_likelihood

def poisson_log_likelihood_scipy(y_true, y_pred):
    """
    Alternative using scipy.stats - more robust
    """
    y_pred = np.maximum(y_pred, 1e-10)
    return np.sum(poisson.logpmf(y_true, y_pred))

# Calculate Poisson deviance
def poisson_deviance(y_true, y_pred):
    """
    Poisson deviance - lower is better
    """
    y_pred = np.maximum(y_pred, 1e-10)
    y_true_safe = np.maximum(y_true, 1e-10)
    
    deviance = 2 * np.sum(
        y_true * np.log(y_true_safe / y_pred) - (y_true - y_pred)
    )
    return deviance

# Calculate all metrics
log_likelihood = poisson_log_likelihood_scipy(y_test, preds_poisson)
deviance = poisson_deviance(y_test, preds_poisson)

# Mean log-likelihood (for interpretability)
mean_log_likelihood = log_likelihood / len(y_test)

print(f"\n=== POISSON REGRESSION BASELINE ===")
print(f"RMSE: {rmse_poisson:.4f}")
print(f"MAE: {mae_poisson:.4f}")
print(f"Log-likelihood: {log_likelihood:.2f}")
print(f"Mean log-likelihood: {mean_log_likelihood:.4f}")
print(f"Poisson deviance: {deviance:.2f}")
print(f"Mean prediction: {preds_poisson.mean():.2f}")
print(f"Mean actual: {y_test.mean():.2f}")

# Additional diagnostic: AIC (Akaike Information Criterion)
n_params = X_train_scaled.shape[1] + 1  # features + intercept
aic = 2 * n_params - 2 * log_likelihood
print(f"AIC: {aic:.2f}")

# Calibration check - compare predicted vs observed rates in bins
def calibration_analysis(y_true, y_pred, n_bins=10):
    """
    Check if predicted rates match observed rates across different ranges
    """
    # Create bins based on predicted values
    bin_edges = np.percentile(y_pred, np.linspace(0, 100, n_bins + 1))
    bin_indices = np.digitize(y_pred, bin_edges) - 1
    bin_indices = np.clip(bin_indices, 0, n_bins - 1)
    
    print(f"\n=== CALIBRATION ANALYSIS ===")
    print("Bin | Predicted | Observed | Count")
    print("-" * 35)
    
    for i in range(n_bins):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            pred_mean = y_pred[mask].mean()
            obs_mean = y_true[mask].mean()
            count = np.sum(mask)
            print(f"{i+1:2d}  | {pred_mean:8.3f} | {obs_mean:7.3f} | {count:4d}")

# Run calibration analysis
calibration_analysis(y_test, preds_poisson)


Training Poisson Regression baseline...

=== POISSON REGRESSION BASELINE ===
RMSE: 1.1880
MAE: 0.9311
Log-likelihood: -988.81
Mean log-likelihood: -1.4937
Poisson deviance: 724.79
Mean prediction: 1.48
Mean actual: 1.49
AIC: 2077.63

=== CALIBRATION ANALYSIS ===
Bin | Predicted | Observed | Count
-----------------------------------
 1  |    0.876 |   0.836 |   67
 2  |    1.059 |   0.894 |   66
 3  |    1.163 |   1.227 |   66
 4  |    1.271 |   1.364 |   66
 5  |    1.371 |   1.379 |   66
 6  |    1.489 |   1.470 |   66
 7  |    1.592 |   1.727 |   66
 8  |    1.735 |   1.894 |   66
 9  |    1.890 |   1.939 |   66
10  |    2.339 |   2.134 |   67
